# Experience with iris dataset using tf.keras & tensorflow

In [0]:
import tensorflow as tf
import pandas as pd
import numpy as np

## Data download and dataset creation witout tf.data

In [0]:
train_ds_url = "http://download.tensorflow.org/data/iris_training.csv"
test_ds_url = "http://download.tensorflow.org/data/iris_test.csv"
ds_columns = ['SepalLength', 'SepalWidth','PetalLength', 'PetalWidth', 'Plants']
species = np.array(['Setosa', 'Versicolor', 'Virginica'], dtype=np.object)

## Load data

In [0]:
categories='Plants'

train_path = tf.keras.utils.get_file(train_ds_url.split('/')[-1], train_ds_url)
test_path = tf.keras.utils.get_file(test_ds_url.split('/')[-1], test_ds_url)
    
train = pd.read_csv(train_path, names=ds_columns, header=0)
train_plantfeatures, train_categories = train, train.pop(categories)

test = pd.read_csv(test_path, names=ds_columns, header=0)
test_plantfeatures, test_categories = test, test.pop(categories)

In [0]:
y_categorical = tf.contrib.keras.utils.to_categorical(train_categories, num_classes=3)
y_categorical_test = tf.contrib.keras.utils.to_categorical(test_categories, num_classes=3)

##  Build  the Dataset
from_tensor_slices

To build the dataset we will use tf.data.Dataset set of elements. 

In [0]:
dataset = tf.data.Dataset.from_tensor_slices((train_plantfeatures, y_categorical))
dataset = dataset.batch(32)
dataset = dataset.shuffle(1000)
dataset = dataset.repeat()

In [0]:
dataset_test = tf.data.Dataset.from_tensor_slices((test_plantfeatures, y_categorical_test))
dataset_test = dataset_test.batch(32)
dataset_test = dataset_test.shuffle(1000)
dataset_test = dataset_test.repeat()

## Build  the Model

In [0]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(16, input_dim=4),
  tf.keras.layers.Dense(3, activation=tf.nn.softmax),
])

model.summary()

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

## Train the Model

In [0]:
model.fit(dataset, steps_per_epoch=32, epochs=100, verbose=1)

## Eval the model

In [0]:
loss, accuracy = model.evaluate(dataset_test, steps=32)

print("loss:%f"% (loss))
print("accuracy: %f"%   (accuracy))

## Use the model

If you need to test another specie, you can modify the **new_specie** array.

In [0]:
new_specie = np.array([7.9,3.8,6.4,2.0])
predition = np.around(model.predict(np.expand_dims(new_specie, axis=0))).astype(np.int)[0]
print("This species should be %s" % species[predition.astype(np.bool)][0])

# Save the model

In [0]:
!mkdir model

In [0]:
tf.keras.models.save_model(
    model,
    "./model/iris_model.h5",
    overwrite=True,
    include_optimizer=True
)

In [0]:
new_model = tf.keras.models.load_model("./model/iris_model.h5")

xarray2 = np.array([7.9,3.8,6.4,2.0])

pred = np.around(new_model.predict(np.expand_dims(xarray2, axis=0))).astype(np.int)[0]

print(pred)

print("That means it's a %s" % species[pred.astype(np.bool)][0])

# Visualize the Graph

In [0]:
graph = tf.get_default_graph()

In [0]:
# Let's visualize our graph!
# Tip: to make your graph more readable you can add a
# name="..." parameter to the individual Ops.

# src: https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/tutorials/deepdream/deepdream.ipynb
# requeried if is not importated before
# import tensorflow as tf
# import numpy as np

from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [0]:
show_graph(graph)